<a href="https://colab.research.google.com/github/xhr0804/BotBuilder-Samples/blob/master/NER_BiLSTM_CRF_BIO2_L2_E128_H128__F1_082.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.train
!wget https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.testa

--2020-12-07 10:37:11--  https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.train
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3281528 (3.1M) [text/plain]
Saving to: ‘eng.train.4’

eng.train.4         100%[===================>]   3.13M  --.-KB/s    in 0.1s    

2020-12-07 10:37:12 (22.8 MB/s) - ‘eng.train.4’ saved [3281528/3281528]

--2020-12-07 10:37:12--  https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.testa
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 827012 (808K) [text/plain]
Savin

In [2]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import time
import matplotlib.pyplot as plt

print(tf.__version__)
# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.3.0


In [3]:
def parse(file):
  left, right = [], []
  for line in open(file):
    line = line.strip()
    if '-DOCSTART-' in line:
      this_left, this_right = [], []
    elif not len(line):
      if len(this_left) > 0:
        left.append(this_left)
        right.append(this_right)
      this_left, this_right = [], []
    else:
      splitted = line.split()
      this_left.append(splitted[0])
      this_right.append(splitted[-1])
  
  new_right = []
  for sentence in right:
    cur = []
    for i in range(len(sentence)):
      if "B-" in sentence[i] or sentence[i] == "O":
        cur.append(sentence[i])
      else: #"I-" in sentence[i]:
        word = sentence[i]
        if i == 0 or sentence[i] != sentence[i-1]:
          cur.append(word.replace("I-", "B-"))
        else:
          cur.append(word)
    new_right.append(cur)
  return left, new_right

In [4]:
left_train, right_train = parse('eng.train')
left_test, right_test = parse('eng.testa')

In [5]:
def countWord(sentences):
  word_count = {}
  for sentence in sentences:
    for word in sentence:
      if word in word_count:
        word_count[word] += 1
      else:
        word_count[word] = 1
  return word_count

In [6]:
word_count = countWord(left_train)
tag_count = countWord(right_train)
sorted_word_count = sorted(word_count.items(), key=lambda x: x[1], reverse=True)
word2idx = {kv[0]: index+2 for index, kv in enumerate(sorted_word_count)}
word2idx["<PAD>"] = 0
word2idx["<UNK>"] = 1
idx2word = {idx:word for word, idx in word2idx.items()}
sorted_tag_count = sorted(tag_count.items(), key=lambda x: x[1], reverse=True)
tag2idx = {kv[0]: index for index, kv in enumerate(sorted_tag_count)}
idx2tag = {idx:tag for tag, idx in tag2idx.items()}

In [7]:
tag2idx.items()

dict_items([('O', 0), ('B-LOC', 1), ('B-PER', 2), ('B-ORG', 3), ('I-PER', 4), ('I-ORG', 5), ('B-MISC', 6), ('I-LOC', 7), ('I-MISC', 8)])

In [8]:
print(left_train[0:5])
print(right_train[0:5])

[['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], ['Peter', 'Blackburn'], ['BRUSSELS', '1996-08-22'], ['The', 'European', 'Commission', 'said', 'on', 'Thursday', 'it', 'disagreed', 'with', 'German', 'advice', 'to', 'consumers', 'to', 'shun', 'British', 'lamb', 'until', 'scientists', 'determine', 'whether', 'mad', 'cow', 'disease', 'can', 'be', 'transmitted', 'to', 'sheep', '.'], ['Germany', "'s", 'representative', 'to', 'the', 'European', 'Union', "'s", 'veterinary', 'committee', 'Werner', 'Zwingmann', 'said', 'on', 'Wednesday', 'consumers', 'should', 'buy', 'sheepmeat', 'from', 'countries', 'other', 'than', 'Britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.']]
[['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O'], ['B-PER', 'I-PER'], ['B-LOC', 'O'], ['O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-LOC', 'O

In [9]:
def convertWord2Idx(words, map, threshold):
  ret = []
  for word in words:
    if word in map and map[word] < threshold:
      ret.append(map[word])
    else:
      ret.append(map["<UNK>"])
  return np.array(ret, dtype=np.int32)

In [10]:
x_train = np.array([convertWord2Idx(words, word2idx, 10000) for words in left_train])
y_train = np.array([convertWord2Idx(words, tag2idx, 10000) for words in right_train])
x_test = np.array([convertWord2Idx(words, word2idx, 10000) for words in left_test])
y_test = np.array([convertWord2Idx(words, tag2idx, 10000) for words in right_test])

In [11]:
x_train = keras.preprocessing.sequence.pad_sequences(x_train,
                                                     padding='post',
                                                     value = word2idx["<PAD>"],
                                                     maxlen = 64)
y_train = keras.preprocessing.sequence.pad_sequences(y_train,
                                                     padding='post',
                                                     value = 0,
                                                     maxlen = 64)

In [12]:
# https://www.tensorflow.org/addons/api_docs/python/tfa/text/crf_log_likelihood
# https://www.tensorflow.org/addons
import tensorflow_addons as tfa

In [13]:
model = keras.Sequential([
  keras.layers.Embedding(10000, 128, mask_zero=True),
  keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True)),
  keras.layers.Dropout(0.5),
  keras.layers.Dense(128, activation='relu'),
  keras.layers.Dense(len(tag2idx.items()))
])

In [14]:
model.add_weight(name="transitions", 
                 shape=(len(tag2idx.items()), len(tag2idx.items())), 
                 dtype=tf.float32, 
                 initializer=tf.keras.initializers.GlorotUniform())

<tf.Variable 'transitions:0' shape=(9, 9) dtype=float32, numpy=
array([[-0.27811137,  0.3763078 , -0.4913741 ,  0.46707165, -0.574925  ,
        -0.44215795, -0.45741367,  0.4145292 , -0.54109406],
       [-0.31678775,  0.05242509, -0.53458357, -0.29504782, -0.41492012,
        -0.03981555,  0.19980872, -0.45985273, -0.21395871],
       [-0.21069184,  0.22937936,  0.28624243, -0.0548476 , -0.5075646 ,
        -0.27818516,  0.5640502 ,  0.36302292,  0.25202787],
       [-0.0132913 ,  0.05199283,  0.5367137 , -0.5280339 , -0.42855406,
         0.4209835 ,  0.25930685,  0.49346137, -0.20463574],
       [-0.1948519 , -0.0402599 , -0.43723404,  0.39831984, -0.02660412,
        -0.2753709 ,  0.5366422 ,  0.52082455,  0.30017048],
       [-0.43503082, -0.5149741 , -0.30065918,  0.47053993,  0.3983739 ,
        -0.05076903, -0.36545494, -0.01554215, -0.07540649],
       [-0.36876848,  0.16070247, -0.04943848,  0.46042192, -0.45872757,
         0.04581881,  0.38788158,  0.1953308 , -0.19240502]

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         1280000   
_________________________________________________________________
bidirectional (Bidirectional (None, None, 128)         98816     
_________________________________________________________________
dropout (Dropout)            (None, None, 128)         0         
_________________________________________________________________
dense (Dense)                (None, None, 128)         16512     
_________________________________________________________________
dense_1 (Dense)              (None, None, 9)           1161      
Total params: 1,396,570
Trainable params: 1,396,570
Non-trainable params: 0
_________________________________________________________________


In [16]:
class CustomLoss(tf.keras.losses.Loss):
    def __init__(
        self,
        transition_params = None,
        reduction: str = tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE,
        name: str = "crf_loss",
    ):
        super().__init__(reduction=reduction, name=name)
        self.transition_params = transition_params
    def call(self, y_true, y_pred):
      seqlen = tf.math.count_nonzero(y_pred._keras_mask, axis=1)
      loglikelihood, _ = tfa.text.crf_log_likelihood(y_pred, y_true, seqlen, self.transition_params)
      return tf.reduce_mean(-loglikelihood)

In [17]:
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [18]:
model.compile(optimizer='adam', 
              loss = CustomLoss(model.trainable_variables[-1]), #if the model output doesn't have softmax, should set from_logits=True
              metrics=['accuracy'])

In [19]:
# m.layers[0].get_weights() get layers variable
model.fit(
    x = x_train[:12000], # batchsize * sentence_len(word index)
    y = y_train[:12000], # batchsize * sentence_len(tag index)
    batch_size = 64,
    epochs = 40,
    validation_data = (x_train[12000:], y_train[12000:]),
    callbacks = [earlystopping]
)

Epoch 1/40
188/188 [==============================] - 26s 139ms/step - loss: 2.3379 - accuracy: 0.8280 - val_loss: 1.5006 - val_accuracy: 0.8768
Epoch 2/40
188/188 [==============================] - 24s 127ms/step - loss: 0.9127 - accuracy: 0.9014 - val_loss: 0.8964 - val_accuracy: 0.9332
Epoch 3/40
188/188 [==============================] - 24s 127ms/step - loss: 0.4487 - accuracy: 0.9525 - val_loss: 0.6301 - val_accuracy: 0.9537
Epoch 4/40
188/188 [==============================] - 23s 124ms/step - loss: 0.2719 - accuracy: 0.9708 - val_loss: 0.5554 - val_accuracy: 0.9614
Epoch 5/40
188/188 [==============================] - 23s 125ms/step - loss: 0.1946 - accuracy: 0.9784 - val_loss: 0.5731 - val_accuracy: 0.9607
Epoch 6/40
188/188 [==============================] - 24s 125ms/step - loss: 0.1482 - accuracy: 0.9826 - val_loss: 0.5921 - val_accuracy: 0.9613
Epoch 7/40
188/188 [==============================] - 23s 125ms/step - loss: 0.1154 - accuracy: 0.9860 - val_loss: 0.6226 - val_ac

In [20]:
def decode(pred_tag_index, idx2tag, sentence):
  ret = [sentence[i] + "(" + idx2tag[pred_tag_index[i]] + ")" for i in range(len(sentence))]
  print(" ".join(ret))

In [21]:
x_test = keras.preprocessing.sequence.pad_sequences(x_test,
                                                     padding='post',
                                                     value = word2idx["<PAD>"],
                                                     maxlen = 64)

In [22]:
y_test = keras.preprocessing.sequence.pad_sequences(y_test,
                                                     padding='post',
                                                     value = 0,
                                                     maxlen = 64)

In [23]:
x_potential = model.predict(x_test)
seqlen = np.array([min(len(sentence), 64) for sentence in right_test])
pred, _ = tfa.text.crf_decode(x_potential, model.trainable_variables[-1], seqlen)
pred = pred.numpy()

In [24]:
from sklearn.metrics import classification_report

In [25]:
pred_label = []
for i in range(len(right_test)):
  pred_label += [idx2tag[pred[i][j]] for j in range(min(len(right_test[i]), 64))]
real_label = []
for i in range(len(right_test)):
  real_label += [idx2tag[y_test[i][j]] for j in range(min(len(right_test[i]), 64))]

In [26]:
print(classification_report(pred_label, real_label))

              precision    recall  f1-score   support

       B-LOC       0.83      0.91      0.87      1660
      B-MISC       0.78      0.86      0.82       838
       B-ORG       0.72      0.84      0.78      1156
       B-PER       0.83      0.84      0.84      1775
       I-LOC       0.74      0.74      0.74       257
      I-MISC       0.66      0.77      0.71       295
       I-ORG       0.71      0.80      0.75       668
       I-PER       0.86      0.89      0.87      1229
           O       0.99      0.97      0.98     43297

    accuracy                           0.96     51175
   macro avg       0.79      0.85      0.82     51175
weighted avg       0.96      0.96      0.96     51175

